[View in Colaboratory](https://colab.research.google.com/github/MariamMohamedFawzy/Deep-Learning-Notebooks/blob/master/Network_in_Network.ipynb)

In [0]:
import tensorflow as tf

# NIN + dropout

In [0]:
tf.reset_default_graph()

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])

#1
w1 = tf.Variable(tf.random_normal(shape=[5, 5, 3, 192], mean=0.0, stddev=0.05))
b1 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.05))
conv1 = tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='SAME')
conv1 = tf.nn.relu(conv1 + b1)


w2 = tf.Variable(tf.random_normal(shape=[1, 1, 192, 160], mean=0.0, stddev=0.05))
b2 = tf.Variable(tf.random_normal((1, 160), mean=0.0, stddev=0.1))
conv2 = tf.nn.conv2d(conv1, w2, strides=[1, 1, 1, 1], padding='SAME')
conv2 = tf.nn.relu(conv2 + b2)

w3 = tf.Variable(tf.random_normal(shape=[1, 1, 160, 96], mean=0.0, stddev=0.05))
b3 = tf.Variable(tf.random_normal((1, 96), mean=0.0, stddev=0.05))
conv3 = tf.nn.conv2d(conv2, w3, strides=[1, 1, 1, 1], padding='SAME')
conv3 = tf.nn.relu(conv3 + b3)

pool1 = tf.layers.max_pooling2d(conv3, 3, 2)

dropout1 = tf.nn.dropout(pool1, 0.5)

#2
w4 = tf.Variable(tf.random_normal(shape=[5, 5, 96, 192], mean=0.0, stddev=0.05))
b4 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.05))
conv4 = tf.nn.conv2d(dropout1, w4, strides=[1, 1, 1, 1], padding='SAME')
conv4 = tf.nn.relu(conv4 + b4)


w5 = tf.Variable(tf.random_normal(shape=[1, 1, 192, 192], mean=0.0, stddev=0.05))
b5 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.05))
conv5 = tf.nn.conv2d(conv4, w5, strides=[1, 1, 1, 1], padding='SAME')
conv5 = tf.nn.relu(conv5 + b5)

w6 = tf.Variable(tf.random_normal(shape=[1, 1, 192, 192], mean=0.0, stddev=0.05))
b6 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.05))
conv6 = tf.nn.conv2d(conv5, w6, strides=[1, 1, 1, 1], padding='SAME')
conv6 = tf.nn.relu(conv6 + b6)

pool2 = tf.layers.max_pooling2d(conv6, 3, 2)

dropout2 = tf.nn.dropout(pool2, 0.5)

#3

w7 = tf.Variable(tf.random_normal(shape=[3, 3, 192, 192], mean=0.0, stddev=0.05))
b7 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.05))
conv7 = tf.nn.conv2d(dropout2, w7, strides=[1, 1, 1, 1], padding='SAME')
conv7 = tf.nn.relu(conv7 + b7)


w8 = tf.Variable(tf.random_normal(shape=[1, 1, 192, 192], mean=0.0, stddev=0.05))
b8 = tf.Variable(tf.random_normal((1, 192), mean=0.0, stddev=0.1))
conv8 = tf.nn.conv2d(conv7, w8, strides=[1, 1, 1, 1], padding='SAME')
conv8 = tf.nn.relu(conv8 + b8)

w9 = tf.Variable(tf.random_normal(shape=[1, 1, 192, 10], mean=0.0, stddev=0.05))
b9 = tf.Variable(tf.random_normal((1, 10), mean=0.0, stddev=0.1))
conv9 = tf.nn.conv2d(conv8, w9, strides=[1, 1, 1, 1], padding='SAME')
conv9 = tf.nn.relu(conv9 + b9)


avgpool = tf.reduce_mean(conv9, axis=[1, 2])


In [6]:
avgpool

<tf.Tensor 'Mean:0' shape=(?, 10) dtype=float32>

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=avgpool))

global_step = tf.Variable(0, trainable=False)

learning_rate = tf.train.exponential_decay(0.001, global_step,
                                           100000, 0.96, staircase=True)

train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

In [0]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, axis=1), tf.argmax(avgpool, axis=1)), tf.float32))

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [0]:
saver = tf.train.Saver()

In [0]:
!mkdir my_cifar10_model

In [23]:

n_epochs = 100
batch_size = 32

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(n_epochs):

    total_accuracy = 0
    for j in range(x_train.shape[0] // batch_size):
      temp_x = x_train[min((j) * batch_size, x_train.shape[0]) : min((j + 1) * batch_size, x_train.shape[0])]
      temp_y = y_train[min((j) * batch_size, x_train.shape[0]) : min((j + 1) * batch_size, x_train.shape[0])]
          
      _, l, acc = sess.run([train_op, loss, accuracy], feed_dict={x: temp_x, y: temp_y})
      
      total_accuracy += acc * batch_size
      
    total_accuracy = total_accuracy / x_train.shape[0]
      
    print("epoch", i + 1, "train accuracy", total_accuracy, ", valid accuracy", sess.run(accuracy, 
                            feed_dict={x: x_test[:1000], 
                                       y: y_test[:1000]}))
    saver.save(sess, './my_cifar10_model',global_step=global_step)
            

epoch 1 train accuracy 0.27562 , valid accuracy 0.351
epoch 2 train accuracy 0.37902 , valid accuracy 0.399
epoch 3 train accuracy 0.4265 , valid accuracy 0.456
epoch 4 train accuracy 0.45804 , valid accuracy 0.471
epoch 5 train accuracy 0.48584 , valid accuracy 0.503
epoch 6 train accuracy 0.51054 , valid accuracy 0.542
epoch 7 train accuracy 0.5277 , valid accuracy 0.544
epoch 8 train accuracy 0.54402 , valid accuracy 0.554
epoch 9 train accuracy 0.55732 , valid accuracy 0.568
epoch 10 train accuracy 0.56844 , valid accuracy 0.568
epoch 11 train accuracy 0.5799 , valid accuracy 0.56
epoch 12 train accuracy 0.58986 , valid accuracy 0.587
epoch 13 train accuracy 0.59698 , valid accuracy 0.603
epoch 14 train accuracy 0.604 , valid accuracy 0.605
epoch 15 train accuracy 0.60946 , valid accuracy 0.599
epoch 16 train accuracy 0.6181 , valid accuracy 0.612
epoch 17 train accuracy 0.62364 , valid accuracy 0.616
epoch 18 train accuracy 0.62808 , valid accuracy 0.632
epoch 19 train accuracy 0.

In [62]:
!ls my_cifar10_model-

ls: cannot access 'my_cifar10_m': No such file or directory


In [24]:
with tf.Session() as sess:
  new_saver = tf.train.import_meta_graph('my_cifar10_model-1608.meta')
  new_saver.restore(sess, tf.train.latest_checkpoint('./'))
  batch_size = 32
  model_accuracy = 0
  for j in range(x_test.shape[0] // batch_size):
      temp_x = x_test[min((j) * batch_size, x_test.shape[0]) : min((j + 1) * batch_size, x_test.shape[0])]
      temp_y = y_test[min((j) * batch_size, x_test.shape[0]) : min((j + 1) * batch_size, x_test.shape[0])]

      acc = sess.run(accuracy, 
                      feed_dict={x: temp_x, 
                                 y: temp_y})
      model_accuracy += acc * temp_x.shape[0]
  model_accuracy = model_accuracy / x_test.shape[0]
  print("test accuracy", model_accuracy)

INFO:tensorflow:Restoring parameters from ./my_cifar10_model-0
test accuracy 0.6801


In [0]:

n_epochs = 100
batch_size = 32

with tf.Session() as sess:
  new_saver = tf.train.import_meta_graph('my_cifar10_model-1608.meta')
  new_saver.restore(sess, tf.train.latest_checkpoint('./'))

  for i in range(n_epochs):

    total_accuracy = 0
    for j in range(x_train.shape[0] // batch_size):
      temp_x = x_train[min((j) * batch_size, x_train.shape[0]) : min((j + 1) * batch_size, x_train.shape[0])]
      temp_y = y_train[min((j) * batch_size, x_train.shape[0]) : min((j + 1) * batch_size, x_train.shape[0])]
          
      _, l, acc = sess.run([train_op, loss, accuracy], feed_dict={x: temp_x, y: temp_y})
      
      total_accuracy += acc * batch_size
      
    total_accuracy = total_accuracy / x_train.shape[0]
      
    print("epoch", i + 1, "train accuracy", total_accuracy, ", valid accuracy", sess.run(accuracy, 
                            feed_dict={x: x_test[:1000], 
                                       y: y_test[:1000]}))
    saver.save(sess, './my_cifar10_model',global_step=global_step)
            

INFO:tensorflow:Restoring parameters from ./my_cifar10_model-0
epoch 1 train accuracy 0.75764 , valid accuracy 0.685
epoch 2 train accuracy 0.75762 , valid accuracy 0.695
epoch 3 train accuracy 0.75888 , valid accuracy 0.678
epoch 4 train accuracy 0.7582 , valid accuracy 0.693
epoch 5 train accuracy 0.7582 , valid accuracy 0.696
epoch 6 train accuracy 0.75982 , valid accuracy 0.696
epoch 7 train accuracy 0.75918 , valid accuracy 0.69
epoch 8 train accuracy 0.76056 , valid accuracy 0.691
epoch 9 train accuracy 0.76064 , valid accuracy 0.704
epoch 10 train accuracy 0.75992 , valid accuracy 0.7
epoch 11 train accuracy 0.7607 , valid accuracy 0.693
epoch 12 train accuracy 0.76126 , valid accuracy 0.686
epoch 13 train accuracy 0.76182 , valid accuracy 0.707
epoch 14 train accuracy 0.76268 , valid accuracy 0.705
epoch 15 train accuracy 0.76264 , valid accuracy 0.691
epoch 16 train accuracy 0.76298 , valid accuracy 0.693
